# Duale Cocktails {#sec-cocktails-duality}

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/Cocktails-Duality.ipynb)

- [Cocktails (shared)](https://febunisofia-my.sharepoint.com/:x:/g/personal/amarov_feb_uni-sofia_bg/EX4kuOiIItRKv7b50LypddwBT3Nj6qwpZNkvfCgh3tXPdQ?e=nJ8lA9&nav=MTVfezk5RUQyQzY2LTRGMDQtNEEyMC05MkVELUIxRjc2NDA2RENEMH0)
- [Cocktails (download)](https://github.com/febse/data/raw/refs/heads/main/opt/Cocktails_2d.xlsx)

:::{.callout-important}
## Excel

In der Übung haben wir den Verbrauch von Ressourcen mit der Funktionen `MMULT` und `TRANSPOSE` berechnet. Das hat an einigen
Rechnern nicht geklappt. Ein möchlicher Grund dazu ist, dass die Excel-Versionen unterschiedlich waren.

Hier finden Sie eine [Anleitung](https://support.microsoft.com/en-us/office/transpose-function-ed039415-ed8a-4a81-93e9-4b6dfac76027) zur Benutzung von `TRANSPOSE`.

Sie können auch `MMULT` mit `TRANSPOSE` auch in der online Version von Excel benutzen: [Cocktails (shared)](https://febunisofia-my.sharepoint.com/:x:/g/personal/amarov_feb_uni-sofia_bg/EX4kuOiIItRKv7b50LypddwBT3Nj6qwpZNkvfCgh3tXPdQ?e=nJ8lA9&nav=MTVfezk5RUQyQzY2LTRGMDQtNEEyMC05MkVELUIxRjc2NDA2RENEMH0).

Da jeder auf diese Onlinedatei zugreifen kann, benutzen Sie bitte den Worksheet mit Ihrem Namen.

:::


In [6]:
%pip install gurobipy plotly sympy

import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


## Barmanagement

Stellen wir uns vor, dass wir eine kleine Bar managen. Wir sind so spezialisiert, dass wir nur Bloody Mary in zwei Varianten anbieten:

- _Bloody Mary Light (BML)_, der aus 20 ml Vodka und 80 ml Tomatensaft besteht
- _Bloody Mary Stark (BMS)_, der aus 40 ml Vodka und 60 ml Tomatensaft besteht

In einer Stunde haben 2000 ml Vodka und 4000 ml Tomatensaft zur Verfügung. Wie viele von beiden Cocktails sollten wir zubereiten, um den Gewinn zu maximieren? Ein Glass Bloody Mary Light (100 ml) bring uns 3 Euro Gewinn ein, ein Glass (100 ml) Bloody Mary Stark bringt uns 4 Euro Gewinn ein.

- Schreiben Sie das Modell zuerst auf Papier auf
- Lösen Sie das Modell mit Excel Solver
- Lösen Sie das Modell mit Gurobi in Python (einmal mit `.addVar` und einmal mit `.addMVar`)
- Formulieren Sie die duale Optimierungsaufgabe und lösen Sie diese mit Excel Solver und Gurobi (auch mit `.addVar` und `.addMVar`)

:::{.callout-note}
## Das Modell (konkret)

Es seien $x_1$ die Anzahl der Cocktails BML und $x_2$ die Anzahl der Cocktails BMS. Dann ist das Modell:

$$
\begin{align*}
\text{max} & \quad 3x_1 + 4x_2  & \text{Gewinn} \\
\text{s.t.} & \quad 20x_1 + 40x_2 \leq 2000 & \text{Vodka}\\
& \quad 80x_1 + 60x_2 \leq 4000 & \text{Tomatensaft} \\
& \quad x_1, x_2 \geq 0
\end{align*}
$$

:::

:::{.callout-note}
## Das Modell (allgemein)

Das Modell hat zwei Zielvariablen: $x_1$ und $x_2$.

Die Paramter des Modells sind:

1. Die Anzahl der Variablen ist $n = 2$
1. Die Koeffizienten der Variablen in der Zielfunktion: $c_1 = 3$, $c_2 = 4$
2. Die rechten Seiten der Restriktionen: $b_1 = 2000$, $b_2 = 4000$
3. Die Koeffizienten der Variablen in den Restriktionen: $a_{11} = 20$, $a_{12} = 40$, $a_{21} = 80$, $a_{22} = 60$

Etwas allgemeiner, sieht das Modell so aus:

$$
\begin{align*}
\text{max} & \quad c_1 x_1 + c_2 x_2 \\
\text{s.t.} & \quad a_{11} x_1 + a_{12} x_2 \leq b_1 \\
& \quad a_{21} x_1 + a_{22} x_2 \leq b_2 \\
& \quad x_1, x_2 \geq 0
\end{align*}
$$

Oder noch kompakter mit Summation:

$$
\begin{align*}
\text{max} & \quad \sum_{j=1}^n c_j x_j \\
\text{s.t.} & \quad \sum_{j=1}^n a_{ij} x_j \leq b_i, \quad i = 1, 2 \\
& \quad x_j \geq 0, \quad j = 1, 2
\end{align*}
$$

:::

:::{.callout-note}
## Excel

In der Excelumsetzung haben wir die Variablen $x_1$ und $x_2$ in den Zellen `B2` und `C2` definiert. 

Die Koeffizienten der Zielfunktion sind in den Zellen `B3` und `C3` gespeichert. 

Der Gewinn in abhängigkeit von den Variablen ist in der Zelle `D3` gespeichert und ist berechnet 
mit der Formel `=SUMPRODUCT($B$2:$C$2, B3:C3)`. Sie können auch die Formel `=$B$2*$B$3+C2*C3` benutzen, allerdings ist `SUMPRODUCT` kompakter.

Die maximal verfügbaren Mengen $b_1 = 2000, b_2 = 4000$ sind in den Zellen `E7` und `E8` gespeichert.

Die Verbrauchskoeffizienten $a_{ij}$ sind in den Zellen `E7:E8` gespeichert.

In den Zellen `D7` und `D8` haben wir den Verbrauch von Vodka und Tomatensaft in Abhängigkeit von den Variablen $x_1$ und $x_2$ gespeichert. In der ersten Variante berechnen wir diese mit der Formel 
`=SUMPRODUCT($B$2:$C$2, B7:C8)`.

In der zweiten Variante möchten wir den Verbrauch mit *einer* Formal berechnen. Das machen wir mit 
`=MMULT(B14:C15,TRANSPOSE(B2:C2))`. Die Transponierung ist notwendig, weil die Matrixmultiplikation nur funktioniert, wenn die Dimensionen der Matrizen übereinstimmen.

:::


Wir möchten nun dieselben Berechnung mit Python und insbesondere mit `numpy` durchführen. Genauso wie wir die Parameter der Aufgabe in Excel Zellen gespeichert haben, werden wir sie in Python in Variablen speichern. In Python speichern wir die Paramter in Arrays (ähnlich zu einem Range in Excel).

Was wir im Folgenden machen ist:

- Wir speichern die Koeffizienten der Zielfunktion in einem Array `gewinn_pro_cocktail`: äquivalent zu den Zellen `B3` und `C3` in Excel
- Wir speichern die rechten Seiten der Restriktionen in einem Array `max_ressourcen`: äquivalent zu den Zellen `E7` und `E8` in Excel
- Wir speichern die Verbrauchskoeffizienten in einer Matrix (Tabelle mit Zahlen) `verbrauchs_koeffizienten`: äquivalent zu den Zellen `B7:C8` in Excel

In [7]:
max_ressourcen = np.array([2000, 4000]) # ml Vodka, ml Tomatensaft

print("Ressourcen:", max_ressourcen)

gewinn_pro_cocktail = np.array([3, 4])

print("Gewinn pro Cocktail:", gewinn_pro_cocktail)

verbrauchs_koeffizienten = np.array([
    [20, 40], # Die Koeffizienten für Vodka (B7:C7)
    [80, 60] # Die Koeffizienten für Tomatensaft (B8:C8)
])

Ressourcen: [2000 4000]
Gewinn pro Cocktail: [3 4]


Wenn wir schon die Koeffizienten eingegeben haben, können wir dieselben Berechnungen wie in Excel durchführen. In Excel haben wir ausser der Eingabe der Parameter nur drei Zellen, die von den anderen bestimmt werden:

- Der Gewinn (D3)
- Der Verbrauch von Vodka und Tomatensaft (D7 und D8)

Wir werden ein Array `x` erstellen (äquivalent zu den Zellen `B2` und `C2` in Excel), das die Anzahl der Cocktails BML und BMS speichert. Zuerst setzen wir die zwei Variablen auf beliebige Werte, z.B. 2 und 3, was ein Produktionsplan mit 2 Cocktails BML und 3 Cocktails BMS bedeutet.



In [8]:
# Produktionsplan x1, x2

x = np.array([2, 3])

# Gewinn bei Produktionsplan x

gewinn_pro_cocktail @ x

np.int64(18)

:::{.callout-important}
## Vergleich mit Excel

Setzen Sie die Werte in den Zellen `B2` und `C2` auf 2 und 3 und vergleichen Sie den Gewinn in der Zelle `D3` mit dem Gewinn, den Sie oben in Python berechnet haben. Sie müssen den gleichen Gewinn bekommen. 

:::

Was der `@` Operator in Python macht, ist Matrixmultiplikation:

$$
\begin{pmatrix}
3 & 4 
\end{pmatrix}
\begin{pmatrix}
2 \\
3
\end{pmatrix}
=
3 \cdot 2 + 4 \cdot 3 = 6 + 12 = 18
$$


Jetzt möchten wir sehen, wie viel Vodka und Tomatensaft wir verbrauchen, wenn wir 2 Cocktails BML und 3 Cocktails BMS produzieren. 

In [ ]:
# Verbrauch an Vodka bei Produktionsplan x (B7): die erste Zeile der Verbrauchskoeffizienten multipliziert mit x

verbrauchs_koeffizienten[0] @ x

np.int64(160)

Das funktioniert genau wie die Berechnung des Gewinns, der einzige Unterschied ist es, dass wir die Verbrauchskoeffizienten für Vodka nehmen (20 und 40)

$$
\begin{pmatrix}
20 & 40 \\
\end{pmatrix}
\begin{pmatrix}
2 \\
3
\end{pmatrix}
=
20 \cdot 2 + 40 \cdot 3 = 40 + 120 = 160
$$


In [ ]:
# Verbrauch an Tomatensaft bei Produktionsplan x (B8): die zweite Zeile der Verbrauchskoeffizienten multipliziert mit x
verbrauchs_koeffizienten[1] @ x

np.int64(340)


Den Verbrauch von Tomatensaft berechnen wir mit den Verbrauchskoeffizienten für Tomatensaft (80 und 60):

$$
\begin{pmatrix}
80 & 60 \\
\end{pmatrix}
\begin{pmatrix}
2 \\
3
\end{pmatrix}
=
80 \cdot 2 + 60 \cdot 3 = 160 + 180 = 340
$$


Nun m;chten wir den Verbrauch von Vodka und Tomatensaft auf einmal berechnen. Das machen wir mit der Matrixmultiplikation. Dieses ist äquivalent zu der Formel `=MMULT(B14:C15,TRANSPOSE(B2:C2))` in Excel. Hier brauchen wir jedoch `x` nicht zu transponieren.

In [11]:
verbrauchs_koeffizienten @ x

array([160, 340])

Was wir berechnet haben ist:

$$
\underset{\text{Operationen}}{\begin{pmatrix}
20 & 40 \\
80 & 60
\end{pmatrix}}
\underset{\text{Daten}}{\begin{pmatrix}
2 \\
3
\end{pmatrix}}
=
\underset{\text{Ergebnis}}{\begin{pmatrix}
20 \cdot 2 + 40 \cdot 3 \\
80 \cdot 2 + 60 \cdot 3
\end{pmatrix}}
=
\begin{pmatrix}
160 \\
340
\end{pmatrix}
$$

Vergleichen Sie die Ergebnisse mit den Werten in den Zellen `D7` und `D8`, sowie `D13` und `D14` in Excel. Sie sollten die gleichen Werte bekommen. 


Berechnen Sie den Verbrauch von Vodka und Tomatensaft für die Produktion von 

- 2 Cocktails BML und 3 Cocktails BMS
- 3 Cocktails BML und 2 Cocktails BMS
- 4 Cocktails BML und 1 Cocktails BMS
- 1 Cocktails BML und 4 Cocktails BMS

Benutzen Sie dazu Formeln in Excel und `numpy` in Python

In [7]:
# Modell erstellen

m = gp.Model("Cocktail")
m.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

# Variablen definieren

x1 = m.addVar(name="Light") # Anzahl von Bloody Mary Light Cocktails
x2 = m.addVar(name="Stark") # Anzahl von Bloody Mary Stark Cocktails

# Zielfunktion definieren

m.setObjective(3 * x1 + 4 * x2, GRB.MAXIMIZE)

# Nebenbedingungen definieren


# m.optimize()

m.write("cocktail.lp") # Speichern des Modells in einer Datei

with open("cocktail.lp", "r") as f:
    print(f.read())

# Ergebnisse ausgeben

# print("Optimaler Cocktailmix:")

# print(f"Bloody Mary Light: {x1.X:.2f} Stück")
# print(f"Bloody Mary Stark: {x2.X:.2f} Stück")

# # Die Nebenbedingungen ausgeben als pandas DataFrame

# # Die Nebenbedingungen ausgeben als pandas DataFrame
# constr_df = pd.DataFrame([
#     (c.ConstrName, c.Pi, c.Slack, c.RHS) for c in m.getConstrs()],
#     columns=["Name", "Schattenpreis", "Slack", "RHS"]
# )
# constr_df

Restricted license - for non-production use only - expires 2026-11-23


Set parameter LogToConsole to value 0


\ Model Cocktail
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  3 Light + 4 Stark
Subject To
Bounds
End



In [8]:
# Dasselbe in kürzerer Form

m1 = gp.Model("Cocktail 2")
m1.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

x = m1.addMVar(2, name="Cocktails")

# Zielfunktion definieren

# Nebenbedingungen definieren


# m1.optimize()

# # Ergebnisse ausgeben
# print("Optimaler Cocktailmix:")
# print(f"Bloody Mary Light: {x.X[0]:.2f} Stück")
# print(f"Bloody Mary Stark: {x.X[1]:.2f} Stück")

# m1.write("cocktail-2.lp")

# with open("cocktail-2.lp") as f:
#     print(f.read())

# # Die Nebenbedingungen ausgeben als pandas DataFrame
# constr_df = pd.DataFrame([
#     (c.ConstrName, c.Pi, c.Slack, c.RHS) for c in m1.getConstrs()],
#     columns=["Name", "Schattenpreis", "Slack", "RHS"]
# )
# constr_df

Set parameter LogToConsole to value 0


## Das duale Problem

Schauen wir uns eine einfache Optimierungsaufgabe an:

$$
\begin{align*}
\text{max} & \quad 2x_1 + 3x_2 \\
\text{s.t.} & \quad x_1 + x_2 \leq 8 \\
& \quad x_1 + 2 x_2 \leq 12 \\
& x_1, x_2 \geq 0
\end{align*}
$$

Ihre Lösung ist $[x_1, x_2] = [4, 4]$, also erreichen wir den grösstmöglichen Gewinn von $2 \cdot 4 + 3 \cdot 4 = 20$ Euro. Können wir aber **beweisen**, dass es keinen grösseren Gewinn gibt?
 

Die duale Aufgabe zur Barmanagement-Optimierung ist:

$$
\begin{align*}
\text{min} & \quad 2000 y_1 + 4000 y_2 \\
\text{s.t.} & \quad 20 y_1 + 80 y_2 \geq 3 \\
& \quad 40 y_1 + 60 y_2 \geq 4 \\
& \quad y_1, y_2 \geq 0 \\
\end{align*}
$$



In [9]:
# Das duale Problem (1)

md = gp.Model("Cocktail dual")
md.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

y1 = md.addVar(name="Wert von Vodka") # Schattenpreis Vodka
y2 = md.addVar(name="Wert von Tomatensaft") # Schattenpreis Tomatensaft

# Einschränkungen definieren


# Kontrolle: das LP ausgeben
md.write("cocktail-dual.lp") # Speichern des Modells in einer Datei

with open("cocktail-dual.lp", "r") as f:
    print(f.read())

# md.optimize()

# # Ergebnisse ausgeben
# print("Optimale Werte der Ressourcen:")

# print(f"Wert von Vodka: {y1.X:.2f} Euro /ml")
# print(f"Wert von Tomatensaft: {y2.X:.2f} Euro/ml")
# print(f"Gewinn: {md.ObjVal:.2f} Euro")

# # Die Nebenbedingungen ausgeben als pandas DataFrame
# constr_df = pd.DataFrame([
#     (c.ConstrName, c.Slack, c.RHS, c.Pi, c.Sense) for c in md.getConstrs()], 
#     columns=["Name", "Slack", "RHS", "Schattenpreis", "Sense"
# ])

# constr_df


Set parameter LogToConsole to value 0


\ Model Cocktail dual
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0 Wert_von_Vodka + 0 Wert_von_Tomatensaft
Subject To
Bounds
End



In [10]:
# Das Duale Problem (2)

m2 = gp.Model("Cocktail Dual 2")

m2.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

# Variablen definieren (mit addMVar)

# Zielfunktion definieren


# Nebenbedingungen definieren (mit addConstr)

# m2.optimize()

m2.write("cocktail-dual-2.lp") # Speichern des Modells in einer Datei

with open("cocktail-dual-2.lp", "r") as f:
    print(f.read())

# # Ergebnisse ausgeben
# print("Schattenpreise:")

# print(f"Wert von Vodka: {y.X[0]:.2f} Euro/ml")
# print(f"Wert von Tomatensaft: {y.X[1]:.2f} Euro/ml")
# print(f"Gewinn: {m2.ObjVal:.2f} Euro")
# # Die Nebenbedingungen ausgeben als pandas DataFrame

# constr_df = pd.DataFrame([
#     (c.ConstrName, c.Slack, c.RHS, c.Pi, c.Sense) for c in m2.getConstrs()],
#     columns=["Name", "Slack", "RHS", "Schattenpreis", "Sense"
# ])

# constr_df


Set parameter LogToConsole to value 0


\ Model Cocktail Dual 2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
 
Subject To
Bounds
End



In [ ]:
# Wie viel von den Ressourcen verbrauchen wir, falls wir 2 BML und 2 BLS machen?

bar_rezepte @ np.array([1, 1])

array([100, 100])

# Cocktails mit drei Zutaten


In [12]:
# Ingredients 100 ml vodka, 100 ml rum, 100 ml tomato juice
zutaten_3 = np.array([1, 1, 1])

# Rezepte (in ml)

bloody_mary_rezept = np.array([20, 0, 80])
rum_pure_rezept = np.array([0, 50, 0])
crazy_rezept = np.array([40, 40, 20])

alle_rezepte = np.stack([
    bloody_mary_rezept,
    rum_pure_rezept,
    crazy_rezept
])

rezept_namen = [
    "Bloody Mary Rezept",
    "Rum Pure Rezept",
    "Crazy Cocktail Rezept"
]

cocktail_colors = [
    "firebrick",
    "steelblue",
    "orange"
]

# Create a 3D scatter plot with plotly showing the rows of all_cocktails as vectors 

fig = go.Figure()

# Add each cocktail as a vector
for i, cocktail in enumerate(alle_rezepte):
    fig.add_trace(go.Scatter3d(
        x=[0, cocktail[0]],  # Start at origin (0, 0, 0)
        y=[0, cocktail[1]],
        z=[0, cocktail[2]],
        mode='lines+markers',
        name=rezept_namen[i],
        line=dict(color=cocktail_colors[i], width=5),
        marker=dict(size=5)
    ))

# Set plot layout
fig.update_layout(
    title="3D Diagram der Cocktails",
    scene=dict(
        xaxis=dict(
            title="Vodka",
            range=[0, 1]
        ),
        yaxis=dict(
            title="Rum",
            range=[0, 1]
        ),
        zaxis=dict(
            title="Tomatensaft",
            range=[0, 1]
        )
    )
)

# Show the plot
fig.show()

## Modifikation von Rezepten


In [13]:

# Nimm die Zutaten, erhöhe die erste Zutat (Vodka) um 20 Prozent, ignoriere alle anderen Zutaten
nimm_den_vodka_verdoppele_ihn_und_ignoriere_den_rest = np.array([2, 0, 0])

# Nimm die zweite Zutat (Rum), verändere die Menge nicht, ignoriere alle anderen Zutaten
nimm_nur_den_rum_so_wie_er_ist = np.array([0, 1, 0])

# Nimm die dritte Zutat (Tomato Juice), verändere die Menge nicht, ignoriere alle anderen Zutaten
nimm_den_tomatensaft_so_wie_er_ist = np.array([0, 0, 1])

In [14]:
verdoppele_den_vodka_im_rezept = np.stack([
    nimm_den_vodka_verdoppele_ihn_und_ignoriere_den_rest,
    nimm_nur_den_rum_so_wie_er_ist,
    nimm_den_tomatensaft_so_wie_er_ist
])

verdoppele_den_vodka_im_rezept

array([[2, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [15]:
modifizierte_rezepte = verdoppele_den_vodka_im_rezept @ alle_rezepte.T
print(alle_rezepte)
modifizierte_rezepte.T

[[20  0 80]
 [ 0 50  0]
 [40 40 20]]


array([[40,  0, 80],
       [ 0, 50,  0],
       [80, 40, 20]])

In [16]:
# Create a 3D scatter plot with plotly showing the rows of all_cocktails as vectors 

# Add each cocktail as a vector
for i, cocktail in enumerate(modifizierte_rezepte.T):
    fig.add_trace(go.Scatter3d(
        x=[0, cocktail[0]],  # Start at origin (0, 0, 0)
        y=[0, cocktail[1]],
        z=[0, cocktail[2]],
        mode='lines+markers',
        name=f"{rezept_namen[i]} modifiziert",
        line=dict(color=cocktail_colors[i], width=5, dash='dash'),
        marker=dict(size=5)
    ))

fig.update_layout(
    title = "3D Diagram der Cocktails: die originalen Rezepte und die modifizierten Rezepte",
)
# Show the plot
fig.show()

In [17]:
bloody_mary_mit_doppelt_vodka_rezept = verdoppele_den_vodka_im_rezept @ bloody_mary_rezept
bloody_mary_mit_doppelt_vodka_rezept

array([40,  0, 80])

In [18]:
crazy_mit_doppelt_vodka_rezept = crazy_rezept @ verdoppele_den_vodka_im_rezept
crazy_mit_doppelt_vodka_rezept

array([80, 40, 20])

In [19]:
switch_vodka_and_tomato = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
])
switch_vodka_and_tomato

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])